In [1]:
# necessary libraries for prediciton
import utils
import pandas as pd
import numpy as np
import pprint

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Useful Functions

In [2]:
def pretty_matrix(matrix, row_label, col_label):
    """Pretty print of the given matrix """

    # Restraining labels that are too big
    row_label = [el[:10] + '..' if len(el) > 10 else el
                for el in row_label]
    col_label = [el[:10] + '..' if len(el) > 10 else el
                for el in col_label]

    # Stringfying everything & Joining top label
    s_matrix = [list([" "] + (col_label))] + \
               [[row_label[row_idx]] + \
                [str(e) for e in row] for row_idx, row in enumerate(matrix)]

    # Length of each matrix column
    len_s = [max(map(len, col)) for col in zip(*s_matrix)]

    # Cell formatation
    formatation = '\t'.join('{{:{}}}'.format(x) for x in len_s)

    # Apply cell formation to each matrix element
    pretty_mat = [formatation.format(*row) for row in s_matrix]

    # Print Pretty Matrix
    print('\n'.join(pretty_mat))


def display_confusion_matrix(values):
    '''Display the given array as a confusion matrix'''
    pretty_matrix([values[0:2], values[2:4]],
                  ['Actual NO', 'Actual YES'],
                  ['Predic NO', 'Predic YES'])

In [3]:
def apply_PCA(df, variance_val=0.9, debug=True):
    '''Apply the PCA algorithm to given dataframe,
    using the given variance val to trim the df'''
    # Necessary to normalize all data to use PCA
    scaler=StandardScaler()
    X_scaled=scaler.fit_transform(df)

    # PCA - keep, by default mode, 90% variance
    pca = PCA(variance_val)    
    pca.fit(X_scaled)
    X_pca = pca.transform(X_scaled)

    if debug:
        ex_variance=np.var(X_pca,axis=0)
        ex_variance_ratio = ex_variance/np.sum(ex_variance)
        print(' > Impact in total variance of each generated feature by PCA:')
        print(ex_variance_ratio)

    principal_df = pd.DataFrame(data = X_pca, index = df.reset_index()['loan_id'])
    
    return (principal_df, pca)

In [4]:
def auc_scorer(y_true, y_pred):
    '''Scorer of Area Under Curve value'''
    fpr, tpr, _ = metrics.roc_curve(y_true, y_pred)
    return metrics.auc(fpr, tpr)

# Prediction Algorithms

* Logistic Regression
* Decision Tree
* Random Forest
* Gradient Boosting

In [5]:
def create_LR():
    '''Create a Logistic Regression model'''
    return LogisticRegression()

In [6]:
def create_DT():
    '''Create a new Decision Tree'''
    # Useful DecisionTree tutorial:
    # https://www.datacamp.com/community/tutorials/decision-tree-classification-python
    return DecisionTreeClassifier()

In [7]:
def create_RF():
    '''Create a new Ranfom Forest model'''
    return RandomForestClassifier()

In [8]:
def create_GB():
    '''Create a new Gradient Boosting model'''
    return GradientBoostingClassifier()

# Prediction

* Predictions are done in this notebook.
* It is also useful to compare how serveral algorithms perform against one another.

In [9]:
# Useful Macros
K_FOLD_NUM_SPLITS = 5
SEED = 42
USE_PCA = False
UNDERSAMPLE = False
UNDERSAMPLE_RATIO = 0.3

# Pretty printer
pp = pprint.PrettyPrinter(indent=4)

In [10]:
dataset =  utils.read_csv_to_df('dataset/preprocessed_data.csv')

if UNDERSAMPLE:
    print(' > Apllying undersampling:')
    entries_df = len(dataset.index)

    # Getting all minor class cases into final dataset
    minor_df = dataset[dataset['status'] == -1]
    num_minor = len(minor_df.index)
    
    print('\t> Classes initial ratio: %f - %f\n\t> Dataset size: %i' %
         (num_minor / entries_df, (entries_df - num_minor) / entries_df, entries_df))

    # Selecting equal number from major class
    major_df = dataset[dataset['status'] == 1].sample(n=int((num_minor / UNDERSAMPLE_RATIO) - num_minor),
                                                      random_state=SEED)
    num_major = len(major_df.index)
    total_under = num_minor + num_major
    
    print('\t> Classes final ratio: %f - %f\n\t> Dataset size: %i' % 
          (num_minor / total_under, num_major / total_under, total_under))

    # Concatenting to main dataframe
    dataset = pd.concat([minor_df, major_df])

dataset = dataset.set_index('loan_id')
display(dataset)

,date,amount,payments,frequency,balance_mean,balance_max,balance_min,balance_std,last_balance,last_trans,...,ratio entrepeneurs,criminalty_growth,unemploymant_growth,account_age_on_loan,days_since_last_transaction,owner_age_on_loan,max_value_in_account_to_loan_ratio,trans_per_day,income_to_payments_ratio,status
loan_id,,,,,,,,,,,,,,,,,,,,,
5314,0.000000,96396,8033,2,12250.000000,20100.0,1100.0,8330.866301,20100.0,3300.0,...,0.100,-0.001909,0.29,105,15,45.986301,0.208515,0.038095,1.201295,-1
5316,0.004721,165960,4610,1,52083.859459,120512.8,700.0,29122.059454,52208.9,-3419.0,...,0.117,-0.002094,0.52,148,2,24.986301,0.726156,0.250000,1.890727,1
6863,0.018096,127080,2118,1,30060.954167,49590.4,800.0,11520.184451,20272.8,-12000.0,...,0.132,0.000539,0.61,170,7,57.191781,0.390230,0.141176,4.273528,1
5325,0.022817,105804,2939,1,41297.480000,65898.5,1000.0,14151.260443,34307.3,178.1,...,0.135,-0.001739,0.29,185,3,53.323288,0.622836,0.135135,3.616139,1
7240,0.049567,274740,4579,2,57188.211111,122893.1,600.0,25256.665817,41112.9,-30.0,...,0.110,-0.001179,0.73,204,6,15.008219,0.447307,0.132353,1.823761,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6818,0.988198,155616,3242,1,44197.509884,75294.1,200.0,11044.494392,60694.1,-14600.0,...,0.110,-0.002052,0.99,691,1,25.693151,0.483845,0.248915,2.840710,1
5625,0.990559,222180,3703,1,55230.444068,130659.1,800.0,26510.559286,59578.8,-6900.0,...,0.113,0.001531,0.78,382,5,57.780822,0.588078,0.154450,2.388064,-1
6805,0.995279,45024,938,1,41994.907692,63659.3,800.0,13151.510254,38384.3,-17800.0,...,0.081,0.000806,1.12,214,4,39.660274,1.413897,0.182243,11.268923,1


In [11]:
STATUS_COL = dataset.columns.get_loc("status")

In [12]:
# Setting X and Y
X = dataset.iloc[:, 0:STATUS_COL]
y = dataset.iloc[:, [STATUS_COL]]
display(X.head())

if USE_PCA:
    print(' > Applying PCA to X_train:')
    X, pca = apply_PCA(X, debug=True)
    display(X.head())

,date,amount,payments,frequency,balance_mean,balance_max,balance_min,balance_std,last_balance,last_trans,...,unemploymant rate '95,ratio entrepeneurs,criminalty_growth,unemploymant_growth,account_age_on_loan,days_since_last_transaction,owner_age_on_loan,max_value_in_account_to_loan_ratio,trans_per_day,income_to_payments_ratio
loan_id,,,,,,,,,,,,,,,,,,,,,
5314,0.000000,96396,8033,2,12250.000000,20100.0,1100.0,8330.866301,20100.0,3300.0,...,3.38,0.100,-0.001909,0.29,105,15,45.986301,0.208515,0.038095,1.201295
5316,0.004721,165960,4610,1,52083.859459,120512.8,700.0,29122.059454,52208.9,-3419.0,...,1.79,0.117,-0.002094,0.52,148,2,24.986301,0.726156,0.250000,1.890727
6863,0.018096,127080,2118,1,30060.954167,49590.4,800.0,11520.184451,20272.8,-12000.0,...,2.28,0.132,0.000539,0.61,170,7,57.191781,0.390230,0.141176,4.273528
5325,0.022817,105804,2939,1,41297.480000,65898.5,1000.0,14151.260443,34307.3,178.1,...,1.42,0.135,-0.001739,0.29,185,3,53.323288,0.622836,0.135135,3.616139
7240,0.049567,274740,4579,2,57188.211111,122893.1,600.0,25256.665817,41112.9,-30.0,...,3.79,0.110,-0.001179,0.73,204,6,15.008219,0.447307,0.132353,1.823761


# Hyper Parameter Tunning

## Random Search 1st to apporach the best solution, GridSearch to refine it

In [13]:
def getLogisticRegressionBest(X, y, debug=True):
    '''Get the Logistic Regression Hyper Parameters'''

    # Maximum number of levels in tree
    max_depth = [int(x) for x in range(2, 20, 4)]
    max_depth.append(None)

    # Create the random grid
    grid = {'penalty': ['l2', 'none'],
            'C': [0.01, 0.05, 0.1, 0.2, 0.5, 1],
            'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
            'class_weight': ["balanced", None]}
    
    if debug:
        pp.pprint(grid)
    
    # Using the grid search for best hyperparameters
    lr = create_LR()
    lr_grid = GridSearchCV(estimator = lr,
                           param_grid = grid,
                           scoring=metrics.make_scorer(auc_scorer,
                                                       greater_is_better=True),
                           cv=K_FOLD_NUM_SPLITS,
                           verbose=2,
                           n_jobs = -1)

    # Fit the grid search model
    lr_grid = lr_grid.fit(X, y)
    
    if debug:
        print('Best Score: ', lr_grid.best_score_)
        print('Best Params: ', lr_grid.best_params_)
        
    # Return score, method & params tuple
    return (lr_grid.best_score_, 'Logistic Regression', lr_grid.best_params_)

In [14]:
def getDecisionTreeBest(X, y, debug=True):
    '''Get the Decision Tree Hyper Parameters'''

    # Maximum number of levels in tree
    max_depth = [int(x) for x in range(2, 20, 4)]
    max_depth.append(None)

    # Create the random grid
    grid = {'criterion': ['gini', 'entropy'],
            'splitter': ['best', 'random'],
            'max_features': ['auto', 'sqrt'],
            'max_depth': max_depth,
            'min_samples_split':  [2, 5, 10],
            'min_samples_leaf':  [1, 2, 4],
            'class_weight': ["balanced", None]}
    
    if debug:
        pp.pprint(grid)
    
    # Using the grid search for best hyperparameters
    dt = create_DT()
    dt_grid = GridSearchCV(estimator = dt,
                           param_grid = grid,
                           scoring=metrics.make_scorer(auc_scorer,
                                                       greater_is_better=True),
                           cv=K_FOLD_NUM_SPLITS,
                           verbose=2,
                           n_jobs = -1)

    # Fit the grid search model
    dt_grid = dt_grid.fit(X, y)
    
    if debug:
        print('Best Score: ', dt_grid.best_score_)
        print('Best Params: ', dt_grid.best_params_)
        
    # Return score, method & params tuple
    return (dt_grid.best_score_, 'Decision Tree', dt_grid.best_params_)

In [15]:
def getRandomForestBest(X, y, debug=True):
    '''Get the Random Forest Hyper Parameters'''

    # Maximum number of levels in tree
    max_depth = [int(x) for x in range(2, 16, 4)]
    max_depth.append(None)

    # Create the random grid
    grid = {'n_estimators': [int(x) for x in range(2, 14, 2)],
            'max_features': ['auto', 'sqrt'],
            'max_depth': max_depth,
            'criterion': ['gini', 'entropy'],
            'min_samples_split':  [2, 5, 10],
            'min_samples_leaf':  [1, 2, 4],
            'bootstrap': [True, False],
            'class_weight': ["balanced", "balanced_subsample", None]}
    
    if debug:
        pp.pprint(grid)
    
    # Using the grid search for best hyperparameters
    rf = create_RF()
    rf_grid = GridSearchCV(estimator = rf,
                           param_grid = grid,
                           scoring=metrics.make_scorer(auc_scorer,
                                                       greater_is_better=True),
                           cv=K_FOLD_NUM_SPLITS,
                           verbose=2,
                           n_jobs = -1)

    # Fit the grid search model
    rf_grid = rf_grid.fit(X, y)
    
    if debug:
        print('Best Score: ', rf_grid.best_score_)
        print('Best Params: ', rf_grid.best_params_)
        
    # Return score, method & params tuple
    return (rf_grid.best_score_, 'Random Forest', rf_grid.best_params_)

In [16]:
def getGradientBoostBest(X, y, debug=True):
    '''Get the Gradient Boost Hyper Parameters'''

    # Create the grid parameters
    grid = {'n_estimators': [int(x) for x in range(2, 14, 2)],
            'learning_rate': [0.1, 0.3, 0.5, 0.7],
            'loss': ['deviance', 'exponential'],
            'criterion': ['friedman_mse', 'mse', 'mae'],
            'min_samples_split':  [2, 5, 10],
            'min_samples_leaf':  [1, 2, 4],
            'random_state': [SEED]}
    
    if debug:
        pp.pprint(grid)
    
    # Using the grid search for best hyperparameters
    gb = create_GB()
    gb_grid = GridSearchCV(estimator = gb,
                           param_grid = grid,
                           scoring=metrics.make_scorer(auc_scorer,
                                                       greater_is_better=True),
                           cv=K_FOLD_NUM_SPLITS,
                           verbose=2,
                           n_jobs = -1)

    # Fit the grid search model
    gb_grid = gb_grid.fit(X, y)
    
    if debug:
        print('Best Score: ', gb_grid.best_score_)
        print('Best Params: ', gb_grid.best_params_)
        
    # Return score, method & params tuple
    return (gb_grid.best_score_, 'Gradient Boosting', gb_grid.best_params_)

In [17]:
# Getting the best algorithm
algorithms = [getLogisticRegressionBest(X, y),
              getDecisionTreeBest(X, y),
              getRandomForestBest(X, y),
              getGradientBoostBest(X, y)]
algorithms.sort(reverse=True, key=lambda el: el[0])

for index, entry in enumerate(algorithms):
    print('%i. %s - %f\n---------' % (index + 1, entry[1], entry[0]))
    
print('Best algorithm: %s' % algorithms[0][1])

{   'C': [0.01, 0.05, 0.1, 0.2, 0.5, 1],
    'class_weight': ['balanced', None],
    'penalty': ['l2', 'none'],
    'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']}
Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 282 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:   15.9s finished
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/edgarcarneiro/Documents/Universit

Best Score:  0.7027609674592987
Best Params:  {'C': 0.01, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'lbfgs'}
{   'class_weight': ['balanced', None],
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 6, 10, 14, 18, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'splitter': ['best', 'random']}
Fitting 5 folds for each of 864 candidates, totalling 4320 fits


[Parallel(n_jobs=-1)]: Done 344 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 2280 tasks      | elapsed:   14.7s
[Parallel(n_jobs=-1)]: Done 4320 out of 4320 | elapsed:   24.9s finished
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best Score:  0.7738258883998616
Best Params:  {'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 6, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'splitter': 'best'}
{   'bootstrap': [True, False],
    'class_weight': ['balanced', 'balanced_subsample', None],
    'criterion': ['gini', 'entropy'],
    'max_depth': [2, 6, 10, 14, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [2, 4, 6, 8, 10, 12]}
Fitting 5 folds for each of 6480 candidates, totalling 32400 fits


[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 1176 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 2800 tasks      | elapsed:   28.0s
[Parallel(n_jobs=-1)]: Done 5006 tasks      | elapsed:   55.7s
[Parallel(n_jobs=-1)]: Done 6294 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 8074 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 10182 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 12610 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 15366 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 18442 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 21846 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 25570 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 29622 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 32400 out of 32400 | elapsed:  7.2min finished
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packa

Best Score:  0.8391116256087372
Best Params:  {'bootstrap': True, 'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 2, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 8}
{   'criterion': ['friedman_mse', 'mse', 'mae'],
    'learning_rate': [0.1, 0.3, 0.5, 0.7],
    'loss': ['deviance', 'exponential'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [2, 4, 6, 8, 10, 12],
    'random_state': [42]}
Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


[Parallel(n_jobs=-1)]: Done 258 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 1468 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done 3498 tasks      | elapsed:   45.2s
[Parallel(n_jobs=-1)]: Done 4831 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 5561 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 6451 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 6473 out of 6480 | elapsed:  3.3min remaining:    0.2s


Best Score:  0.7423266308555941
Best Params:  {'criterion': 'friedman_mse', 'learning_rate': 0.7, 'loss': 'deviance', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 4, 'random_state': 42}
1. Random Forest - 0.839112
---------
2. Decision Tree - 0.773826
---------
3. Gradient Boosting - 0.742327
---------
4. Logistic Regression - 0.702761
---------
Best algorithm: Random Forest


[Parallel(n_jobs=-1)]: Done 6480 out of 6480 | elapsed:  3.3min finished
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Using method with higher score with our data

In [26]:
# Cross validation settings
auc_scores = []
confusion_matrixes = []
cv = KFold(n_splits=K_FOLD_NUM_SPLITS, random_state=SEED, shuffle=False)

# CHANGE THIS LINE TO CHANGE THE USED CLASSIFICATION METHOD
# classifier = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=2, min_samples_split=10, splitter='best')
# classifier = GradientBoostingClassifier(criterion='friedman_mse', learning_rate=0.7, loss='deviance',
#                                         min_samples_leaf=2, min_samples_split=2, n_estimators=8, random_state=SEED)
classifier = RandomForestClassifier(bootstrap=False, class_weight='balanced', criterion='entropy',
                                    max_depth=2, max_features='sqrt', min_samples_leaf=4,
                                    min_samples_split=5, n_estimators=8)


# Applying Cross validation
for train_index, test_index in cv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Training with this fold
    classifier.fit(X_train, y_train)
    
    # Testing & Measuring accuracy
    y_pred = classifier.predict(X_test)
    
    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
    auc_scores.append(metrics.auc(fpr, tpr))
    confusion_matrixes.append(metrics.confusion_matrix(y_test, y_pred).ravel())

/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/ipykernel_launcher.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/ipykernel_launcher.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/ipykernel_launcher.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/ipykernel_launcher.py:21: DataConversionW

In [27]:
# Printing the obtained results
print('Classification Method used:', classifier, '\n')
print('AUC scores:', auc_scores)
print('> Average: ', sum(auc_scores)/len(auc_scores))
for cf in confusion_matrixes:
    display_confusion_matrix(cf)

Classification Method used: RandomForestClassifier(bootstrap=False, class_weight='balanced',
                       criterion='entropy', max_depth=2, max_features='sqrt',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=4,
                       min_samples_split=5, min_weight_fraction_leaf=0.0,
                       n_estimators=8, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False) 

AUC scores: [0.6432748538011696, 0.7685185185185185, 0.8512931034482758, 0.8050314465408805, 0.7833333333333333]
> Average:  0.7702902511284355
          	Predic NO	Predic YES
Actual NO 	4        	5         
Actual YES	9        	48        
          	Predic NO	Predic YES
Actual NO 	8        	4         
Actual YES	7        	47        
          	Predic NO	Predic YES
Actual NO 	7        	1         
Actual YES	10       	48        
          	Predic NO	Predic YES
Actual NO 

### After having our model trained we shall use the model on the data to be sumitted in the kaggle

In [20]:
test_dataset =  utils.read_csv_to_df('dataset/test_dataset.csv')
test_dataset = test_dataset.set_index('loan_id')
display(test_dataset.head())

,date,amount,payments,frequency,balance_mean,balance_max,balance_min,balance_std,last_balance,last_trans,...,ratio entrepeneurs,criminalty_growth,unemploymant_growth,account_age_on_loan,days_since_last_transaction,owner_age_on_loan,max_value_in_account_to_loan_ratio,trans_per_day,income_to_payments_ratio,status
loan_id,,,,,,,,,,,,,,,,,,,,,
5895,0.000000,93960,1566,1,54520.202247,88246.7,800.0,13768.555214,49548.5,-19700.0,...,0.132,0.000539,0.61,452,2,34.616438,0.939194,0.196903,5.651767,NaN
7122,0.001420,260640,7240,1,31518.182051,88731.8,-718.6,20629.925861,11565.4,12284.0,...,0.100,0.000479,0.15,490,3,47.986301,0.340438,0.159184,1.240389,NaN
6173,0.007102,232560,4845,0,40175.612500,79286.6,200.0,15672.003047,45754.0,-2692.0,...,0.135,-0.001739,0.29,630,1,57.742466,0.340930,0.279365,2.119573,NaN
6142,0.025568,221880,3698,1,44440.912676,74216.8,1000.0,13853.751837,38913.4,-2872.0,...,0.149,-0.006147,0.58,311,8,53.350685,0.334491,0.228296,2.413715,NaN
5358,0.025568,38520,3210,1,20231.313158,31302.0,900.0,4462.709852,18914.3,-3115.0,...,0.087,0.000104,0.53,597,7,57.167123,0.812617,0.190955,2.693951,NaN


In [21]:
# We now remove the Y column with NaNs
test_dataset = test_dataset.iloc[:, 0:STATUS_COL]
display(test_dataset.head())

,date,amount,payments,frequency,balance_mean,balance_max,balance_min,balance_std,last_balance,last_trans,...,unemploymant rate '95,ratio entrepeneurs,criminalty_growth,unemploymant_growth,account_age_on_loan,days_since_last_transaction,owner_age_on_loan,max_value_in_account_to_loan_ratio,trans_per_day,income_to_payments_ratio
loan_id,,,,,,,,,,,,,,,,,,,,,
5895,0.000000,93960,1566,1,54520.202247,88246.7,800.0,13768.555214,49548.5,-19700.0,...,2.28,0.132,0.000539,0.61,452,2,34.616438,0.939194,0.196903,5.651767
7122,0.001420,260640,7240,1,31518.182051,88731.8,-718.6,20629.925861,11565.4,12284.0,...,1.10,0.100,0.000479,0.15,490,3,47.986301,0.340438,0.159184,1.240389
6173,0.007102,232560,4845,0,40175.612500,79286.6,200.0,15672.003047,45754.0,-2692.0,...,1.42,0.135,-0.001739,0.29,630,1,57.742466,0.340930,0.279365,2.119573
6142,0.025568,221880,3698,1,44440.912676,74216.8,1000.0,13853.751837,38913.4,-2872.0,...,3.39,0.149,-0.006147,0.58,311,8,53.350685,0.334491,0.228296,2.413715
5358,0.025568,38520,3210,1,20231.313158,31302.0,900.0,4462.709852,18914.3,-3115.0,...,2.41,0.087,0.000104,0.53,597,7,57.167123,0.812617,0.190955,2.693951


In [22]:
final_df = pd.DataFrame()

if USE_PCA:
    # Using train PCA and classifying
    scaler=StandardScaler()
    X_test_scaled=scaler.fit_transform(test_dataset)
    predictions_df = pd.DataFrame(data = pca.transform(X_test_scaled),
                                  index=test_dataset.reset_index()['loan_id'])
    display(predictions_df)

    predictions_df['Predicted'] = classifier.predict(predictions_df)
    final_df = predictions_df.reset_index()\
                    [['loan_id', 'Predicted']]\
                    .rename(columns={
                        'loan_id': 'Id'
                    })
    
else:
    final_df = test_dataset.copy()
    final_df['Predicted'] = classifier.predict(final_df)
    final_df = final_df.reset_index()\
                        [['loan_id', 'Predicted']]\
                        .rename(columns={
                            'loan_id': 'Id'
                        })\
                        .drop_duplicates()

display(final_df)

,Id,Predicted
0,5895,1
1,7122,-1
2,6173,1
3,6142,1
4,5358,-1
...,...,...
349,4989,-1
350,5221,-1
351,6402,1
352,5346,-1


In [23]:
# Outputting predictions to .csv
# CHANGE FILE NAME TO PRESERVE DIFFERENT INSTANCES
utils.write_df_to_csv(final_df, 'predictions', 'prediction.csv')